# SignalFlow Dev Tools

Manage your SignalFlow app data directly from VS Code during development.
**This is USER mode** - view and update your tickets, checklists, and progress from the app database.

**Quick Actions:**
- View your sprint tickets with checklists
- Update ticket status and checklist items
- View release checklist from workflow modes
- Manage career progress and test cases

In [ ]:
# Setup - Run this first
import sys
sys.path.insert(0, '..')

import os
os.chdir('..')

import json
from datetime import datetime
from src.app.infrastructure.supabase_client import get_client

client = get_client()
print(f"✅ Setup complete at {datetime.now().strftime('%H:%M:%S')}")
print(f"📊 Supabase: {'Connected' if client else 'Not configured'}")

# Helper to pretty print
def pprint(data):
    if isinstance(data, (dict, list)):
        print(json.dumps(data, indent=2, default=str))
    else:
        print(data)

## 📋 Sprint Tickets & Checklists
View your active sprint tickets with task checklists from the app

In [ ]:
# Load Sprint Tickets from Database
if client:
    result = client.table("tickets").select(
        "id, ticket_id, title, status, priority, sprint_points, task_decomposition, tags"
    ).eq("in_sprint", True).order("created_at", desc=True).execute()
    
    tickets = result.data
    print(f"\n📋 SPRINT TICKETS ({len(tickets)} active)\n" + "="*50)
    
    for ticket in tickets:
        status_icons = {
            'done': '✅', 'in_progress': '🔄', 'in_review': '👁️',
            'todo': '⬜', 'backlog': '📝', 'blocked': '🚫'
        }
        icon = status_icons.get(ticket['status'], '❓')
        
        print(f"\n{icon} [{ticket['ticket_id']}] {ticket['title']}")
        print(f"   Status: {ticket['status']} | Points: {ticket.get('sprint_points', 0)}")
        
        # Show task decomposition checklist
        if ticket.get('task_decomposition'):
            tasks = ticket['task_decomposition']
            if isinstance(tasks, str):
                tasks = json.loads(tasks)
            
            print(f"\n   📝 Checklist:")
            if isinstance(tasks, list):
                for task in tasks:
                    if isinstance(task, dict):
                        done = task.get('done', task.get('completed', False))
                        name = task.get('name', task.get('task', task.get('title', str(task))))
                        check = '✅' if done else '⬜'
                        print(f"      {check} {name}")
                    else:
                        print(f"      ⬜ {task}")
            elif isinstance(tasks, dict):
                for key, val in tasks.items():
                    check = '✅' if val else '⬜'
                    print(f"      {check} {key}")
else:
    print("⚠️ Supabase not connected - configure SUPABASE_URL and SUPABASE_KEY")

## ✏️ Update Ticket Checklist
Mark checklist items as done/undone

In [ ]:
# Update Ticket Checklist Item
def update_checklist(ticket_id: str, task_index: int, done: bool):
    """
    Update a checklist item for a ticket.
    
    Args:
        ticket_id: The ticket_id (e.g. "DEV-123")
        task_index: Index of the task in the checklist (0-based)
        done: True to mark complete, False to unmark
    """
    if not client:
        print("⚠️ Supabase not connected")
        return
    
    # Get current ticket
    result = client.table("tickets").select("id, task_decomposition").eq("ticket_id", ticket_id).single().execute()
    
    if not result.data:
        print(f"❌ Ticket {ticket_id} not found")
        return
    
    tasks = result.data.get('task_decomposition', [])
    if isinstance(tasks, str):
        tasks = json.loads(tasks)
    
    if not tasks or task_index >= len(tasks):
        print(f"❌ Task index {task_index} out of range")
        return
    
    # Update the task
    if isinstance(tasks[task_index], dict):
        tasks[task_index]['done'] = done
    else:
        tasks[task_index] = {'name': tasks[task_index], 'done': done}
    
    # Save back
    client.table("tickets").update({
        "task_decomposition": tasks,
        "updated_at": datetime.now().isoformat()
    }).eq("id", result.data['id']).execute()
    
    status = "✅ Done" if done else "⬜ Undone"
    print(f"{status}: Task {task_index} on {ticket_id}")

# Example usage (uncomment to use):
# update_checklist("DEV-123", 0, True)  # Mark first item done
# update_checklist("DEV-123", 2, False)  # Unmark third item

## 🧪 Test Cases (from App)
View test cases defined in your tickets, not repo pytest files

In [ ]:
# Load Test Cases from Ticket Decompositions
# Test cases are often embedded in task_decomposition or can be in a separate field

if client:
    result = client.table("tickets").select(
        "ticket_id, title, task_decomposition, tags"
    ).eq("in_sprint", True).execute()
    
    print(f"\n🧪 TEST CASES FROM TICKETS\n" + "="*50)
    
    for ticket in result.data:
        tasks = ticket.get('task_decomposition', [])
        if isinstance(tasks, str):
            try:
                tasks = json.loads(tasks)
            except:
                tasks = []
        
        # Look for test-related tasks
        test_tasks = []
        if isinstance(tasks, list):
            for task in tasks:
                task_name = task.get('name', str(task)) if isinstance(task, dict) else str(task)
                if any(kw in task_name.lower() for kw in ['test', 'verify', 'validate', 'check', 'acceptance', 'qa']):
                    test_tasks.append(task)
        
        # Also check tags for test markers
        tags = ticket.get('tags', []) or []
        has_test_tag = any('test' in (t or '').lower() for t in tags)
        
        if test_tasks or has_test_tag:
            print(f"\n📋 [{ticket['ticket_id']}] {ticket['title']}")
            
            for task in test_tasks:
                if isinstance(task, dict):
                    done = task.get('done', False)
                    name = task.get('name', str(task))
                else:
                    done = False
                    name = str(task)
                check = '✅' if done else '⬜'
                print(f"   {check} {name}")
            
            if not test_tasks and has_test_tag:
                print("   ⚠️ Has test tag but no test tasks defined")
    
    print("\n💡 Tip: Add test tasks to task_decomposition with 'test', 'verify', or 'acceptance' in the name")
else:
    print("⚠️ Supabase not connected")

## 🚀 Release Checklist (from Workflow Modes)
View release checklist from the app's workflow configuration

In [ ]:
# Load Release Checklist from Workflow Modes
if client:
    # Get workflow modes from database
    result = client.table("workflow_modes").select(
        "mode_key, name, icon, steps"
    ).eq("is_active", True).order("sort_order").execute()
    
    modes = result.data
    
    # Look for release/deployment related modes
    release_mode = None
    for mode in modes:
        if any(kw in (mode['mode_key'] or '').lower() for kw in ['release', 'deploy', 'ship', 'production']):
            release_mode = mode
            break
    
    print(f"\n🚀 RELEASE CHECKLIST\n" + "="*50)
    
    if release_mode:
        print(f"\n{release_mode.get('icon', '🚀')} {release_mode['name']}")
        steps = release_mode.get('steps', [])
        if isinstance(steps, str):
            steps = json.loads(steps)
        
        for step in steps:
            if isinstance(step, dict):
                name = step.get('label', step.get('name', str(step)))
                done = step.get('completed', step.get('done', False))
            else:
                name = str(step)
                done = False
            check = '✅' if done else '⬜'
            print(f"   {check} {name}")
    else:
        # Show default release checklist if no mode defined
        print("\n⚠️ No release workflow mode found in database")
        print("   Creating default checklist...\n")
        
        default_steps = [
            "All tests passing",
            "Code review complete",
            "Documentation updated",
            "Staging verified",
            "Security scan complete",
            "Performance check done",
            "Rollback plan ready",
            "Stakeholder approval"
        ]
        for step in default_steps:
            print(f"   ⬜ {step}")
        
        print("\n💡 Add a 'release' workflow mode in the app to customize this checklist")
    
    # Show all workflow modes available
    print(f"\n\n📂 Available Workflow Modes:")
    for mode in modes:
        print(f"   {mode.get('icon', '📋')} {mode['name']} ({mode['mode_key']})")
else:
    print("⚠️ Supabase not connected")

In [ ]:
# Update Ticket Status
def update_ticket_status(ticket_id: str, new_status: str):
    """
    Update a ticket's status.
    
    Args:
        ticket_id: The ticket_id (e.g. "DEV-123")
        new_status: One of: backlog, todo, in_progress, in_review, blocked, done
    """
    valid_statuses = ['backlog', 'todo', 'in_progress', 'in_review', 'blocked', 'done']
    
    if new_status not in valid_statuses:
        print(f"❌ Invalid status. Use one of: {', '.join(valid_statuses)}")
        return
    
    if not client:
        print("⚠️ Supabase not connected")
        return
    
    result = client.table("tickets").update({
        "status": new_status,
        "updated_at": datetime.now().isoformat()
    }).eq("ticket_id", ticket_id).execute()
    
    if result.data:
        icons = {'done': '✅', 'in_progress': '🔄', 'todo': '⬜', 'blocked': '🚫'}
        icon = icons.get(new_status, '📋')
        print(f"{icon} {ticket_id} → {new_status}")
    else:
        print(f"❌ Ticket {ticket_id} not found")

# Example usage (uncomment to use):
# update_ticket_status("DEV-123", "in_progress")
# update_ticket_status("DEV-123", "done")

## 📊 Career Progress & Skill Tracking
View your skill development and career memories from the app

In [ ]:
# Load Career Data: Skills & Recent Progress
if client:
    # Get skill tracker data
    skills_result = client.table("skill_tracker").select(
        "skill_name, category, proficiency_level, projects_count, tickets_count, last_used_at"
    ).order("proficiency_level", desc=True).limit(15).execute()
    
    print(f"\n📊 TOP SKILLS\n" + "="*50)
    
    for skill in skills_result.data:
        level = skill['proficiency_level']
        bar = "█" * (level // 10) + "░" * (10 - level // 10)
        print(f"   {skill['skill_name']:20} [{bar}] {level}%")
        if skill.get('projects_count') or skill.get('tickets_count'):
            print(f"   {' '*20}  📁 {skill.get('projects_count', 0)} projects, 🎫 {skill.get('tickets_count', 0)} tickets")
    
    # Get recent career memories
    memories_result = client.table("career_memories").select(
        "memory_type, title, skills, created_at"
    ).order("created_at", desc=True).limit(5).execute()
    
    print(f"\n\n🏆 RECENT ACHIEVEMENTS\n" + "="*50)
    
    type_icons = {
        'completed_project': '🎯',
        'ai_implementation': '🤖',
        'skill_milestone': '📈',
        'achievement': '🏆',
        'learning': '📚'
    }
    
    for memory in memories_result.data:
        icon = type_icons.get(memory['memory_type'], '📋')
        date = memory['created_at'][:10] if memory.get('created_at') else ''
        print(f"   {icon} {memory['title']}")
        if memory.get('skills'):
            print(f"      Skills: {', '.join(memory['skills'][:3])}")
else:
    print("⚠️ Supabase not connected")

In [ ]:
# Recent Meetings with Signals
if client:
    result = client.table("meetings").select(
        "id, meeting_name, meeting_date, signals"
    ).order("created_at", desc=True).limit(5).execute()
    
    print(f"\n📅 RECENT MEETINGS\n" + "="*50)
    
    for meeting in result.data:
        date = meeting.get('meeting_date', '')[:10] if meeting.get('meeting_date') else 'No date'
        print(f"\n   📋 {meeting['meeting_name'][:40]}")
        print(f"      Date: {date}")
        
        signals = meeting.get('signals', {})
        if isinstance(signals, str):
            try:
                signals = json.loads(signals)
            except:
                signals = {}
        
        if signals:
            signal_counts = {}
            for sig_type, sig_list in signals.items():
                if isinstance(sig_list, list):
                    signal_counts[sig_type] = len(sig_list)
            
            if signal_counts:
                counts_str = ', '.join(f"{v} {k}" for k, v in signal_counts.items())
                print(f"      Signals: {counts_str}")
else:
    print("⚠️ Supabase not connected")

## 🔧 Quick App Actions
Update app data without leaving VS Code

In [ ]:
# Add Standup Update
def add_standup(content: str):
    """Submit a standup update to the app."""
    if not client:
        print("⚠️ Supabase not connected")
        return
    
    result = client.table("standup_updates").insert({
        "content": content,
        "sprint_date": datetime.now().strftime('%Y-%m-%d')
    }).execute()
    
    if result.data:
        print(f"✅ Standup submitted for {datetime.now().strftime('%Y-%m-%d')}")
    else:
        print("❌ Failed to submit standup")

# Example usage (uncomment to use):
# add_standup("Working on DEV-123 authentication feature. Blocked by API key setup.")

In [ ]:
# Add Career Memory (completed project, AI implementation, etc.)
def add_career_memory(
    memory_type: str,
    title: str,
    description: str = None,
    skills: list = None
):
    """
    Add a career memory/achievement to track your progress.
    
    Args:
        memory_type: One of: completed_project, ai_implementation, skill_milestone, achievement, learning
        title: Short title for the memory
        description: Optional longer description
        skills: List of skills demonstrated (e.g. ["Python", "FastAPI", "OAuth"])
    """
    valid_types = ['completed_project', 'ai_implementation', 'skill_milestone', 'achievement', 'learning']
    
    if memory_type not in valid_types:
        print(f"❌ Invalid type. Use one of: {', '.join(valid_types)}")
        return
    
    if not client:
        print("⚠️ Supabase not connected")
        return
    
    result = client.table("career_memories").insert({
        "memory_type": memory_type,
        "title": title,
        "description": description,
        "skills": skills or [],
        "is_ai_work": memory_type == 'ai_implementation'
    }).execute()
    
    if result.data:
        icons = {'completed_project': '🎯', 'ai_implementation': '🤖', 'achievement': '🏆'}
        icon = icons.get(memory_type, '📋')
        print(f"{icon} Added: {title}")
        if skills:
            print(f"   Skills: {', '.join(skills)}")
    else:
        print("❌ Failed to add memory")

# Example usage (uncomment to use):
# add_career_memory("completed_project", "OAuth Integration", skills=["OAuth2", "FastAPI", "Security"])
# add_career_memory("ai_implementation", "Signal Extraction Pipeline", skills=["LLM", "Prompt Engineering"])

In [ ]:
# Update Skill Level
def update_skill(skill_name: str, level_delta: int = 5, ticket_count_delta: int = 0):
    """
    Update a skill's proficiency level and/or ticket count.
    
    Args:
        skill_name: Name of the skill (e.g. "Python", "FastAPI")
        level_delta: Amount to increase proficiency (0-100 total)
        ticket_count_delta: Number of tickets to add to count
    """
    if not client:
        print("⚠️ Supabase not connected")
        return
    
    # Check if skill exists
    existing = client.table("skill_tracker").select("id, proficiency_level, tickets_count").eq("skill_name", skill_name).execute()
    
    if existing.data:
        # Update existing skill
        current = existing.data[0]
        new_level = min(100, max(0, current['proficiency_level'] + level_delta))
        new_tickets = current.get('tickets_count', 0) + ticket_count_delta
        
        client.table("skill_tracker").update({
            "proficiency_level": new_level,
            "tickets_count": new_tickets,
            "last_used_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat()
        }).eq("id", current['id']).execute()
        
        print(f"📈 {skill_name}: {current['proficiency_level']}% → {new_level}%")
    else:
        # Create new skill
        client.table("skill_tracker").insert({
            "skill_name": skill_name,
            "proficiency_level": max(0, min(100, level_delta)),
            "tickets_count": max(0, ticket_count_delta),
            "category": "general",
            "last_used_at": datetime.now().isoformat()
        }).execute()
        
        print(f"✅ New skill added: {skill_name} at {level_delta}%")

# Example usage (uncomment to use):
# update_skill("FastAPI", level_delta=5, ticket_count_delta=1)
# update_skill("React Native", level_delta=10)

## 🔀 Git & Deployment Status
Quick view of git and deployment status

In [ ]:
# Git Status
import subprocess

try:
    branch = subprocess.check_output(["git", "branch", "--show-current"], text=True).strip()
    status = subprocess.check_output(["git", "status", "--porcelain"], text=True)
    
    print(f"\n🔀 GIT STATUS\n" + "="*50)
    print(f"   Branch: {branch}")
    
    if status:
        changes = len(status.strip().split('\n'))
        print(f"   Changes: {changes} files")
        print(f"\n{status[:500]}")
    else:
        print("   Working directory clean ✅")
        
    # Recent commits
    log = subprocess.check_output(
        ["git", "log", "--oneline", "-5"],
        text=True
    ).strip()
    
    print(f"\n   Recent commits:")
    for line in log.split('\n'):
        print(f"      {line}")
        
except Exception as e:
    print(f"❌ Git error: {e}")

In [ ]:
# Quick Summary Dashboard
if client:
    print(f"\n📊 QUICK SUMMARY\n" + "="*50)
    
    # Tickets summary
    tickets = client.table("tickets").select("status").eq("in_sprint", True).execute()
    status_counts = {}
    for t in tickets.data:
        s = t['status']
        status_counts[s] = status_counts.get(s, 0) + 1
    
    print(f"\n   🎫 Sprint Tickets:")
    for status, count in sorted(status_counts.items()):
        icon = {'done': '✅', 'in_progress': '🔄', 'todo': '⬜'}.get(status, '📋')
        print(f"      {icon} {status}: {count}")
    
    # Skills count
    skills = client.table("skill_tracker").select("id", count="exact").execute()
    print(f"\n   📊 Skills tracked: {skills.count}")
    
    # Meetings count (this week)
    from datetime import timedelta
    week_ago = (datetime.now() - timedelta(days=7)).isoformat()
    meetings = client.table("meetings").select("id", count="exact").gte("created_at", week_ago).execute()
    print(f"   📅 Meetings this week: {meetings.count}")
    
    # Career memories
    memories = client.table("career_memories").select("id", count="exact").execute()
    print(f"   🏆 Career memories: {memories.count}")
    
    print(f"\n   ⏰ Updated: {datetime.now().strftime('%H:%M:%S')}")
else:
    print("⚠️ Run setup cell first")